In [1]:
! pip install semantic-kernel -U 
! pip install qdrant_client -U 

In [2]:
import os
import json

In [3]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview")
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'azure_chat_competion_service': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x11797ac20>}, chat_services={'azure_chat_competion_service': <function Kernel.add_chat_service.<locals>.<lambda> at 0x11797ab00>}, text_embedding_generation_services={'embeddings_services': <function Kernel.add_text_embedding_generation_service.<locals>.<lambda> at 0x1179d49d0>}, default_text_completion_service='azure_chat_competion_service', default_chat_service='azure_chat_competion_service', default_text_embedding_generation_service='embeddings_services', retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [6]:
base_plugin = "./plugins"

In [7]:
files_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "FilePlugin")

In [8]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [9]:
kblist = []

In [10]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await notesFunc(content)
    print(result) 
    json_result = json.loads(result.result)
    kblist.append(json_result)
    file.close()

{"kb": "History of machine learning", "content": "The history of machine learning and artificial intelligence is marked by significant discoveries and periods of progress and stagnation. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). The concept of a 'machine that can think' was introduced by Alan Turing in 1950. The Dartmouth Summer Research Project in 1956 coined the term 'artificial intelligence'. The period from 1956 to 1974 is known as 'The golden years' of AI. The 'AI Winter' from 1974 to 1980 was a period of reduced funding and confidence in AI. The 1980s saw the rise of 'expert systems'. The 'AI Chill' from 1987 to 1993 was marked by the rise of personal computers. From 1993 to 2011, the field matured with the increase in data and compute power. Today, machine learning and AI are integral parts of our lives."}
{
"kb

In [11]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await transcripsFunc(content)
    print(result)
    json_result = json.loads(result.result)
    for item in json_result:
        kblist.append(item)
    file.close()

[
{"kb": "Alan Turing and the Turing test", "content": "Alan Turing, foundation, machine thinking, Turing test, intelligent computer"},
{"kb": "The Dartmouth Summer Research Project on AI", "content": "Artificial intelligence, Dartmouth College, summer research, AI birth, research field"},
{"kb": "The golden years of AI", "content": "AI optimism, problem solving, natural language processing, micro worlds, intelligent machines"},
{"kb": "The AI winter", "content": "Complexity, understated promise, limited compute power, ethics questions, funding dried"},
{"kb": "Resurgence and fall of AI for expert systems", "content": "Expert systems, AI optimism, practical applications, too specialized, personal computers"},
{"kb": "Growth in AI driven by more data and more powerful hardware", "content": "Compute and storage growth, internet rise, smartphones, large data sets, machine learning"},
{"kb": "Big Data", "content": "Compute power, data sets growth, machine learning, problem solving, everyda

In [12]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by significant discoveries and periods of progress and stagnation. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). The concept of a 'machine that can think' was introduced by Alan Turing in 1950. The Dartmouth Summer Research Project in 1956 coined the term 'artificial intelligence'. The period from 1956 to 1974 is known as 'The golden years' of AI. The 'AI Winter' from 1974 to 1980 was a period of reduced funding and confidence in AI. The 1980s saw the rise of 'expert systems'. The 'AI Chill' from 1987 to 1993 was marked by the rise of personal computers. From 1993 to 2011, the field matured with the increase in data and compute power. Today, machine learning and AI are integral parts of our lives."},
 

In [13]:
base_vectordb = 'aboutMLKBDemoDemo'

In [14]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [15]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [16]:
kernel.register_memory_store(memory_store=qdrant_store)

In [17]:
import uuid

In [18]:
uuid.uuid4()

UUID('9ea759ed-568a-4b5b-b9e5-404fa4d48d53')

In [19]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await kernel.memory.save_information(base_vectordb, id=id, text=content)

In [20]:
ask = "can you tell me what is different ML and AI"

memories = await kernel.memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [21]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text

Top Result: The difference between AI and ML - AI requires human level intelligence, ML subset of AI, ML uses algorithms with score 0.8859153


In [22]:
answer_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "AnswerPlugin")

In [23]:
answerFunc = answer_plugin["Summary"]

In [24]:
summary_result = await answerFunc(result)

In [25]:
summary_result.result

'AI, or Artificial Intelligence, aims to mimic human intelligence. Machine Learning (ML), on the other hand, is a subset of AI that uses algorithms to analyze data, learn from it, and make predictions or decisions. In simple terms, AI is the broader concept of machines being able to carry out tasks in a smart way, while ML is the application of AI where machines learn and improve from experience.'